In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import datetime
import string
import time

# Load datasets
customers_df = pd.read_csv("../Cleaned_Datasets/customer_SG_only.csv", index_col=0)  # customer data
orders_df = pd.read_csv("../Cleaned_Datasets/orders_generated.csv", index_col=0)  # order data
products_df = pd.read_csv("../Cleaned_Datasets/products_cleaned.csv", index_col=0)  # product data

# Convert customer_id columns to strings in both DataFrames
customers_df['customer_id'] = customers_df['customer_id'].astype(str)
orders_df['customer_id'] = orders_df['customer_id'].astype(str)

# Load pre-trained DialoGPT model and tokenizer (LLM integration)
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Context management for follow-up queries
context = {
    "last_product": None,  # Stores the last product mentioned
    "last_intent": None,   # Stores the last intent (price, rating, etc.)
    "last_customer": None, # Stores last customer ID
    "last_order": None     # Stores last order ID
}

# LLM (DialoGPT) Chat Handling
def chat_with_dialoGPT(user_input, chat_history_ids=None):
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids
    attention_mask = torch.ones(bot_input_ids.shape, dtype=torch.long)
    
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        attention_mask=attention_mask,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids

# Function to get customer information
def get_customer_info(customer_id):
    customer_id = str(customer_id)
    customers_df['customer_id'] = customers_df['customer_id'].astype(str)
    customer = customers_df[customers_df['customer_id'] == customer_id]
    
    if not customer.empty:
        last_login_days_ago = customer['last_login_day'].values[0]
        last_login_date = (datetime.datetime.now() - datetime.timedelta(days=int(last_login_days_ago))).strftime('%Y-%m-%d')
        checkout_message = "This customer has never checked out."
        
        last_checkout_days_ago = customer['last_checkout_day'].values[0]
        if last_checkout_days_ago.isdigit():
            last_checkout_date = (datetime.datetime.now() - datetime.timedelta(days=int(last_checkout_days_ago))).strftime('%Y-%m-%d')
            checkout_message = f"and their most recent checkout was {last_checkout_days_ago} days ago on {last_checkout_date}."
        
        context["last_customer"] = customer_id  # Store for future queries
        return f"Customer {customer_id} last logged in {last_login_days_ago} days ago on {last_login_date} {checkout_message}"
    
    return "Could not find the customer. Please check the ID!"


# Function to recommend products based on customer history 
def recommend_products(customer_id):
    customer_id = str(customer_id)  # Ensure customer_id is a string
    
    # Get customer's past orders
    customer_orders = orders_df[orders_df['customer_id'] == customer_id]
    
    if customer_orders.empty:
        # If no orders, select 3 popular products
        product_recommendations = products_df[['title', 'price_actual']].sample(3)
        return "This customer has no past orders. Here are some popular recommendations:\n" + \
               product_recommendations.to_string(index=False)
    
    # Get the list of product IDs that the customer has already purchased
    purchased_product_ids = customer_orders['product_id'].unique()
    
    # Recommend products that the customer hasn't purchased yet
    available_products = products_df[~products_df['product_id'].isin(purchased_product_ids)]
    
    if available_products.empty:
        return "No new product recommendations at the moment."
    
    # Select 3 products from the remaining ones
    recommended_products = available_products.sample(min(3, len(available_products)))
    
    # Format the output
    recommendations = recommended_products[['title', 'price_actual']].to_string(index=False)
    return f"Based on your purchase history, I recommend these products:\n{recommendations}"


# Function to get order status
def get_order_status(order_id):
    order_id = str(order_id)  # Ensure order ID is a string
    orders_df['order_id'] = orders_df['order_id'].astype(str)

    order = orders_df[orders_df['order_id'] == order_id]
    if not order.empty:
        product_id = order['product_id'].values[0]
        order_time = order['order_time'].values[0]
        context["last_order"] = order_id  # Store order ID for follow-up
        return (f"Good news! Order {order_id} was placed on {order_time}. "
                f"The order includes product {product_id}. ")
    else:
        return (f"Oops, it seems there’s no order with ID {order_id}. "
                "Can you please check the ID and try again? I’m here to help!")

# Function to handle product-related queries (price or rating)
def get_product_info(product_name, info_type="price"):
    # Clean up the product name by removing punctuation
    product_name_cleaned = product_name.translate(str.maketrans('', '', string.punctuation))
    
    # Split the user input into individual keywords
    keywords = product_name_cleaned.split()
    
    # Perform a case-insensitive search for rows that contain all keywords in the title
    matching_products = products_df[
        products_df['title'].apply(lambda x: all(kw.lower() in x.lower() for kw in keywords))
    ]
    
    if not matching_products.empty:
        # Get the first match (can be expanded to multiple matches if needed)
        product = matching_products.iloc[0]
        price = product['price_actual']
        rating = product['item_rating']
        title = product['title']
        
        # Store product context for follow-up
        context["last_product"] = title  # Use the product title for follow-up queries
        context["last_intent"] = info_type  # Whether the user asked for price or rating
        
        # Provide appropriate information based on info_type
        if info_type == "price":
            return (f"I found the product you're looking for! The '{title}' is priced at {price}. ")
        elif info_type == "rating":
            return (f"The '{title}' has a rating of {rating} stars.")
    else:
        return f"I couldn't find any product containing the keywords '{product_name}'. Please try different keywords!"
    
# Function to collect customer feedback after the session
def request_feedback():
    feedback = input("Bot: How would you rate your experience on a scale from 1 to 5? ")
    if feedback.isdigit() and 1 <= int(feedback) <= 5:
        return int(feedback)
    return None

# Regex-based intent detection
def detect_intent_and_entities(user_input):
    # Prioritize recommendation detection over customer queries
    if "recommend" in user_input.lower() and "customer" in user_input.lower():
        customer_id = re.search(r'\d+', user_input).group(0)
        return "recommendation", {"customer_id": customer_id}

    # Regex patterns to detect customer query intent
    if re.search(r'customer.*id.*\d+', user_input, re.IGNORECASE):
        intent = "customer_query"
        customer_id = re.search(r'\d+', user_input).group(0)
        return intent, {"customer_id": customer_id}
    
    # Regex patterns to detect order query intent
    if re.search(r'order.*id', user_input, re.IGNORECASE):
        intent = "order_query"
        order_id = re.search(r'order_[0-9]+', user_input, re.IGNORECASE)
        return intent, {"order_id": order_id.group(0)} if order_id else {"order_id": None}
    
    # Regex patterns to detect product price or rating queries
    if re.search(r'price|rating', user_input, re.IGNORECASE):
        product_name_match = re.search(r'(?:price|rating) of (.+)', user_input, re.IGNORECASE)
        product_name = product_name_match.group(1).strip() if product_name_match else None
        intent = "product_price" if "price" in user_input.lower() else "product_rating"
        return intent, {"product_name": product_name}

    # Fallback for general queries
    return "general", {}

# Chatbot core function with LLM and metrics collection
def chatbot():
    print("Hi there! I'm your friendly chatbot, here to help you with any queries. Type 'quit' to exit the chat.")
    chat_history_ids = None  # Initialize chat_history_ids to None at the start
    total_queries = 0  # To track the number of queries handled
    response_times = []  # To track response times for efficiency evaluation
    feedback_scores = []  # To track user feedback ratings
    
    while True:
        user_input = input("You: ").strip().lower()
        
        # Check if the user wants to quit
        if user_input == 'quit':
            # Ask for feedback before quitting
            feedback = request_feedback()
            if feedback:
                feedback_scores.append(feedback)
                print(f"Thank you for your feedback! You rated this session {feedback}/5.")
            
            # Say goodbye and break the loop
            print("Goodbye! Feel free to ask for assistance anytime!")
            break

        # Start the timer to measure response time
        start_time = time.time()

        # Enhanced Natural Language Processing for intent and entity detection
        intent, entities = detect_intent_and_entities(user_input)

        # Handle customer queries
        if intent == "customer_query":
            customer_id = entities.get("customer_id")
            if customer_id:
                response = get_customer_info(customer_id)
            else:
                response = "It seems like you asked about a customer, but I couldn't find the ID. Please provide the customer ID."

        # Handle order queries
        elif intent == "order_query":
            order_id = entities.get("order_id")
            if order_id:
                response = get_order_status(order_id)
            else:
                response = "It seems like you asked about an order, but I couldn't find the order ID. Please provide the order ID."

        # Handle product queries based on intent
        elif intent in ["product_price", "product_rating"]:
            product_name = entities.get("product_name")
            
            if product_name:
                # Update product name in the context when a new product is mentioned
                context["last_product"] = product_name.translate(str.maketrans('', '', string.punctuation))
                context["last_intent"] = "price" if intent == "product_price" else "rating"
                
                # Respond with the appropriate information (price or rating)
                response = get_product_info(product_name, info_type=context["last_intent"])
            else:
                # If product name is not explicitly mentioned, check if there's a follow-up on the last product
                if context.get("last_product"):
                    product_name = context["last_product"]
                    # Update intent based on user input (switch between price or rating)
                    if "price" in user_input:
                        context["last_intent"] = "price"
                    elif "rating" in user_input:
                        context["last_intent"] = "rating"
                    
                    # Provide the appropriate information based on the updated intent
                    response = get_product_info(product_name, info_type=context["last_intent"])
                else:
                    response = "Could you please specify which product you're asking about?"

        # Handle product recommendations
        elif intent == "recommendation":
            customer_id = entities.get("customer_id")
            if customer_id:
                # First, get customer info
                customer_info = get_customer_info(customer_id)
                
                # Then, provide recommendations
                recommendations = recommend_products(customer_id)
                
                # Combine the two responses
                response = f"{customer_info}\n{recommendations}"
            else:
                response = "Please provide a customer ID to get recommendations."

        # Handle follow-up queries on the last product
        elif "price" in user_input and context.get("last_product"):
            # User asks for price after a previous query (like rating)
            product_name = context["last_product"]
            response = get_product_info(product_name, info_type="price")
            context["last_intent"] = "price"  # Update the intent to price

        elif "rating" in user_input and context.get("last_product"):
            # User asks for rating after a previous query (like price)
            product_name = context["last_product"]
            response = get_product_info(product_name, info_type="rating")
            context["last_intent"] = "rating"  # Update the intent to rating
        
        else:
            # Fallback to DialoGPT if no specific intent was detected
            response, chat_history_ids = chat_with_dialoGPT(user_input, chat_history_ids)

        # Stop the timer and calculate the response time
        end_time = time.time()
        response_time = end_time - start_time
        response_times.append(response_time)

        # Track the number of queries
        total_queries += 1

        # Print the chatbot's response
        print(f"Bot: {response}")

    # Calculate average response time and feedback scores for efficiency and satisfaction metrics
    if response_times:
        avg_response_time = sum(response_times) / len(response_times)
        print(f"\nAverage response time: {avg_response_time:.2f} seconds")

    if feedback_scores:
        avg_feedback_score = sum(feedback_scores) / len(feedback_scores)
        print(f"Average feedback rating: {avg_feedback_score:.1f}/5")

# Start the chatbot
chatbot()
